In [3]:
import numpy as np
import random

class SudokuLSGA:
    def __init__(self, puzzle, population_size=100, max_generations=150, pc1=0.9, pc2=0.7, pm1=0.1, pm2=0.05):
        self.puzzle = np.array(puzzle)
        self.population_size = population_size
        self.max_generations = max_generations
        self.pc1 = pc1  # Individual crossover rate
        self.pc2 = pc2  # Row crossover rate
        self.pm1 = pm1  # Swap mutation rate
        self.pm2 = pm2  # Reinitialization mutation rate
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for _ in range(self.population_size):
            individual = self.puzzle.copy()
            for row in range(9):
                missing_numbers = [num for num in range(1, 10) if num not in individual[row]]
                np.random.shuffle(missing_numbers)
                for col in range(9):                    
                    if individual[row, col] == 0:
                        individual[row, col] = missing_numbers.pop()
            population.append(individual)
        return population

    def fitness(self, individual):
        def count_duplicates(arr):
            return len(arr) - len(set(arr))

        row_errors = sum(count_duplicates(row) for row in individual)
        col_errors = sum(count_duplicates(individual[:, col]) for col in range(9))
        block_errors = sum(
            count_duplicates(individual[row:row+3, col:col+3].flatten())
            for row in range(0, 9, 3) for col in range(0, 9, 3)
        )
        return row_errors + col_errors + block_errors

    def crossover(self, parent1, parent2):
        child1, child2 = parent1.copy(), parent2.copy()
        for row in range(9):
            if random.random() < self.pc2:
                child1[row], child2[row] = child2[row], child1[row]
        return child1, child2

    def mutate(self, individual):
        for row in range(9):
            if random.random() < self.pm1:
                indices = [i for i in range(9) if self.puzzle[row, i] == 0]
                if len(indices) > 1:
                    i1, i2 = random.sample(indices, 2)
                    individual[row, i1], individual[row, i2] = individual[row, i2], individual[row, i1]
            if random.random() < self.pm2:
                indices = [i for i in range(9) if self.puzzle[row, i] == 0]
                missing_numbers = [num for num in range(1, 10) if num not in individual[row]]
                if len(missing_numbers) >= len(indices):
                    np.random.shuffle(missing_numbers)
                    for i in indices:
                        individual[row, i] = missing_numbers.pop()
        return individual

    def local_search(self, individual):
        # Column local search
        for col in range(9):
            col_values = individual[:, col]
            duplicates = [num for num in range(1, 10) if list(col_values).count(num) > 1]
            for dup in duplicates:
                rows = [row for row in range(9) if individual[row, col] == dup]
                for row in rows:
                    options = [num for num in range(1, 10) if num not in individual[row]]
                    if options:
                        individual[row, col] = random.choice(options)

        # Sub-block local search
        for row in range(0, 9, 3):
            for col in range(0, 9, 3):
                block = individual[row:row+3, col:col+3]
                duplicates = [num for num in range(1, 10) if list(block.flatten()).count(num) > 1]
                for dup in duplicates:
                    positions = [(r, c) for r in range(3) for c in range(3) if block[r, c] == dup]
                    for r, c in positions:
                        options = [num for num in range(1, 10) if num not in block.flatten()]
                        if options:
                            block[r, c] = random.choice(options)
        return individual

    def evolve(self):
        for generation in range(self.max_generations):
            self.population.sort(key=self.fitness)
            if self.fitness(self.population[0]) == 0:
                print(f"Solution found at generation {generation}")
                return self.population[0]

            new_population = self.population[:self.population_size // 2]
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(self.population[:self.population_size // 2], 2)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.append(child1)
                new_population.append(child2)

            self.population = [self.local_search(ind) for ind in new_population]
        print("Max generations reached without finding a solution.")
        return None

# Example Sudoku Puzzle (0 represents empty cells)
puzzle = [
    [0, 0, 9, 0, 0, 0, 1, 0, 0],
    [2, 1, 7, 0, 0, 0, 3, 6, 8],
    [0, 0, 0, 2, 0, 7, 0, 0, 0],
    [0, 6, 4, 1, 0, 3, 5, 8, 0],
    [0, 7, 0, 0, 0, 0, 0, 3, 0],
    [1, 5, 0, 4, 2, 8, 0, 7, 9],
    [0, 0, 0, 5, 8, 9, 0, 0, 0],
    [4, 8, 5, 0, 0, 0, 2, 9, 3],
    [0, 0, 6, 3, 0, 2, 8, 0, 0]
]

solver = SudokuLSGA(puzzle)
solution = solver.evolve()
if solution is not None:
    print("Solved Sudoku:")
    print(solution)


Max generations reached without finding a solution.


In [5]:
import numpy as np
import random

class SudokuLSGA:
    def __init__(self, puzzle, population_size=100, max_generations=500, pc2=0.7, pm1=0.1, pm2=0.05):
        self.puzzle = np.array(puzzle)
        self.population_size = population_size
        self.max_generations = max_generations
        self.pc2 = pc2  # Row crossover rate
        self.pm1 = pm1  # Swap mutation rate
        self.pm2 = pm2  # Reinitialization mutation rate
        self.population = self.initialize_population()

    def initialize_population(self):
        """Initializes a population by filling empty cells randomly while keeping original numbers fixed."""
        population = []
        for _ in range(self.population_size):
            individual = self.puzzle.copy()
            for row in range(9):
                missing_numbers = [num for num in range(1, 10) if num not in individual[row]]
                np.random.shuffle(missing_numbers)
                for col in range(9):
                    if individual[row, col] == 0:
                        individual[row, col] = missing_numbers.pop()
            population.append(individual)
        return population

    def fitness(self, individual):
        """Computes the fitness function by counting duplicate values in rows, columns, and 3x3 blocks."""
        def count_duplicates(arr):
            unique, counts = np.unique(arr[arr > 0], return_counts=True)
            return sum(count - 1 for count in counts)

        row_errors = sum(count_duplicates(individual[row, :]) for row in range(9))
        col_errors = sum(count_duplicates(individual[:, col]) for col in range(9))
        block_errors = sum(
            count_duplicates(individual[row:row+3, col:col+3].flatten())
            for row in range(0, 9, 3) for col in range(0, 9, 3)
        )
        return row_errors + col_errors + block_errors

    def crossover(self, parent1, parent2):
        """Applies row crossover with a probability pc2."""
        child1, child2 = parent1.copy(), parent2.copy()
        for row in range(9):
            if random.random() < self.pc2:
                # Swap only mutable (non-clue) cells in the row
                mutable_indices = [i for i in range(9) if self.puzzle[row, i] == 0]
                for i in mutable_indices:
                    child1[row, i], child2[row, i] = child2[row, i], child1[row, i]
        return child1, child2

    def mutate(self, individual):
        """Performs swap and reinitialization mutation while ensuring valid Sudoku constraints."""
        for row in range(9):
            mutable_indices = [i for i in range(9) if self.puzzle[row, i] == 0]
            
            if len(mutable_indices) > 1:
                if random.random() < self.pm1:
                    # Swap two random mutable numbers
                    i1, i2 = random.sample(mutable_indices, 2)
                    individual[row, i1], individual[row, i2] = individual[row, i2], individual[row, i1]

                if random.random() < self.pm2:
                    # Reinitialize the mutable cells in the row
                    missing_numbers = [num for num in range(1, 10) if num not in individual[row]]
                    np.random.shuffle(missing_numbers)
                    
                    # Ensure we don't pop from an empty list
                    if len(missing_numbers) >= len(mutable_indices):
                        for i in mutable_indices:
                            individual[row, i] = missing_numbers.pop()
        return individual

    def local_search(self, individual):
        """Performs local optimizations on the individual by reducing duplicate numbers in columns and blocks."""
        # Column local search
        for col in range(9):
            col_values = individual[:, col]
            duplicates = [num for num in range(1, 10) if list(col_values).count(num) > 1]
            for dup in duplicates:
                rows = [row for row in range(9) if individual[row, col] == dup and self.puzzle[row, col] == 0]
                for row in rows:
                    options = [num for num in range(1, 10) if num not in individual[row]]
                    if options:
                        individual[row, col] = random.choice(options)

        # Sub-grid (3x3 block) local search
        for row in range(0, 9, 3):
            for col in range(0, 9, 3):
                block = individual[row:row+3, col:col+3]
                block_flat = block.flatten()
                duplicates = [num for num in range(1, 10) if list(block_flat).count(num) > 1]
                for dup in duplicates:
                    positions = [(r, c) for r in range(3) for c in range(3) if block[r, c] == dup and self.puzzle[row+r, col+c] == 0]
                    for r, c in positions:
                        options = [num for num in range(1, 10) if num not in block_flat]
                        if options:
                            block[r, c] = random.choice(options)
        return individual

    def evolve(self):
        """Runs the evolutionary algorithm to find a Sudoku solution."""
        for generation in range(self.max_generations):
            self.population.sort(key=self.fitness)
            best_fitness = self.fitness(self.population[0])

            if best_fitness == 0:
                print(f"Solution found at generation {generation}")
                return self.population[0]

            new_population = self.population[:self.population_size // 2]  # Keep top 50% elite
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(self.population[:self.population_size // 2], 2)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.append(child1)
                new_population.append(child2)

            self.population = [self.local_search(ind) for ind in new_population]

        print("Max generations reached without finding a solution.")
        return None

# Example Sudoku Puzzle (0 represents empty cells)
puzzle = [
    [0, 0, 9, 0, 0, 0, 1, 0, 0],
    [2, 1, 7, 0, 0, 0, 3, 6, 8],
    [0, 0, 0, 2, 0, 7, 0, 0, 0],
    [0, 6, 4, 1, 0, 3, 5, 8, 0],
    [0, 7, 0, 0, 0, 0, 0, 3, 0],
    [1, 5, 0, 4, 2, 8, 0, 7, 9],
    [0, 0, 0, 5, 8, 9, 0, 0, 0],
    [4, 8, 5, 0, 0, 0, 2, 9, 3],
    [0, 0, 6, 3, 0, 2, 8, 0, 0]
]

solver = SudokuLSGA(puzzle)
solution = solver.evolve()
if solution is not None:
    print("Solved Sudoku:")
    print(solution)


Solution found at generation 15
Solved Sudoku:
[[5 4 9 8 3 6 1 2 7]
 [2 1 7 9 5 4 3 6 8]
 [6 3 8 2 1 7 9 5 4]
 [9 6 4 1 7 3 5 8 2]
 [8 7 2 6 9 5 4 3 1]
 [1 5 3 4 2 8 6 7 9]
 [3 2 1 5 8 9 7 4 6]
 [4 8 5 7 6 1 2 9 3]
 [7 9 6 3 4 2 8 1 5]]
